### Imports

In [ ]:
import os                                  #Hello2021
os.environ["OMP_NUM_THREADS"] = "8"        #"export" OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "8"   #"export" OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "8"        #"export" MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "8" #"export" VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "8"    #Hello2021

In [ ]:
import io
import os
import gzip
import json
import scipy
import random
import warnings
import numpy as np
import numba as nb
import pandas as pd
import datetime as dt
import seaborn as sns
from tqdm import tqdm
from tqdm import trange
import scipy.stats as st
from collections import OrderedDict
import scipy.sparse as sparse
from scipy.linalg import orth
import matplotlib.pyplot as plt
from numpy import linalg as lin
import implicit
import pickle
warnings.filterwarnings('ignore')
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from datetime import datetime as dt_dt
from scipy.sparse.linalg import spsolve
from numpy.linalg import qr as QR_decomp
from scipy.sparse import csr_matrix, find
from pandas.api.types import CategoricalDtype
from typing import List
import torch
import torch.nn.functional as F
from torch import nn
from polara.evaluation.pipelines import random_grid
import bottleneck as bn

In [ ]:
import sys
sys.path.append('/home/oolaleke/incremental_PSI/Scripts/')
#sys.path.append('/content/')

In [ ]:
import a_DataHelpers
import b_AlgoFunctions
import c_PredEval
import d_CorrScores
import e_VAEfunction

In [ ]:
from a_DataHelpers import getPivotMonths,Time_DataSplit,TestTrain_DataSplit,SingleRatingMatrix
from a_DataHelpers import AllRatingMatrices,All_SingleStepRatMat,psiStep_RatMat,psiAllStep_RatMat
from a_DataHelpers import Find_NewUsersItems,adjustedPSI_DF,adjustedAllDF,get_NEWHoldout,ADJUST_mainDF
from a_DataHelpers import  SingleStepRatMat_2,AllSingleStepRatMat_2

from b_AlgoFunctions import integrator,getStartingValues,integratorOnMat,last_psiTrainMat, pureSVD
from b_AlgoFunctions import Updt_RowMatrix,getRow_Mat,row_update, Updt_ColMatrix,getCol_Mat,colunm_update
from b_AlgoFunctions import UsersItems_RatPair,getRowCol_psiupdt,ITEMS_defferredStatus,USERS_defferredStatus
from b_AlgoFunctions import getV_listUpdate, SingleStep_UPDATE,ALLSTEPs_UPDATE
from b_AlgoFunctions import HitrEval_noprint,TuneALS,nonzeros,least_squares_cg,alternating_least_squares_cg

from c_PredEval import TopNPred,TQDMgetALLTopNPred, Hitrate_Eval,getAll_HitRate,mean_confidence_interval
from c_PredEval import TQDMgetALLTopNPred_ALLUSERS,getALLTopNPred_ALLUSERS,get_ALLRandPred
from c_PredEval import topN_Index,getAll_RandomHitRate,getMOSTPOP_Pred,getAll_MostPOPHitRate,getAll_MOSTPOP_Pred
from c_PredEval import StepCoverage_Ratio,AllSteps_Coverage_Ratio,getAll_VAEMRR, MRR_Eval,getAll_MRR_Eval
from c_PredEval import TQDMgetAll_MRR_Eval, getAll_MostPOP_MRR,getAll_RandomRecMRR

from d_CorrScores import no_copy_csr_matrix,build_rank_weights_matrix,rank_weighted_jaccard_index
from d_CorrScores import Updt_getAll_AvgCorr, updtCorr_4AllRanks

from e_VAEfunction import split_users, split_train_test, numerize_data, get_TrainRatMat, getTrainTest_RatMat


In [ ]:
#from c_PredEval import ials_TopNPred,ials_getALLPred,ials_getALLPredTQDM,ials_ALLUSERSpred,
#ialsALLPred_ALLUSERS,TQDM_ialsALLPred_ALLUSERS


## iALS Functions

### Pred & Eval 

In [ ]:
def topN_Index(a, n):
    parted = np.argpartition(a, -n)[-n:]
    return parted[np.argsort(-a[parted])]

def ials_TopNPred(RatingMat,holdout,users_vec,items_vec,user_column, N):  #N == Top_N
    TestUsers = holdout[user_column]            #prediction for holdout users alone
    HOLDOUT_usersMat = RatingMat[TestUsers,:]   #this doubles as the "previously seen items"
    testusers_vec = users_vec[TestUsers, :]
    PVVT =  testusers_vec.dot(items_vec.T) 
    users_column = HOLDOUT_usersMat.nonzero()[0]
    items_column = HOLDOUT_usersMat.nonzero()[1]
    args = np.array([users_column,items_column])
    np.put(PVVT, np.ravel_multi_index(args, PVVT.shape),-np.inf)   #downsample previously seen items
    TopN_pred = np.apply_along_axis(topN_Index, 1,PVVT,n = N)
    return TopN_pred

#####  #
def ials_getALLPred(RatingMat_List,HOLDOUT_list,U_list,V_list,user_column,N):
    All_TOPN_PRED = []
    for RatMat,holdout,users_vec,items_vec in zip(RatingMat_List,HOLDOUT_list,U_list,V_list):
        TopN_pred =  ials_TopNPred(RatMat,holdout,users_vec,items_vec,user_column, N)  
        All_TOPN_PRED.append(TopN_pred)
    return All_TOPN_PRED

def ials_ALLUSERSpred(RatingMat,users_vec,items_vec,N):  ##Prediction for all users ...||Not just Holdout
    PVVT =  users_vec.dot(items_vec.T) 
    users_column = RatingMat.nonzero()[0]
    items_column = RatingMat.nonzero()[1]
    args = np.array([users_column,items_column])
    np.put(PVVT, np.ravel_multi_index(args, PVVT.shape),-np.inf)   ##downsample previously seen items
    TopN_pred = np.apply_along_axis(topN_Index, 1,PVVT,n = N)
    return TopN_pred

def ialsALLPred_ALLUSERS(RatingMat_List,U_list,V_list,N):
    All_TOPN_PRED = []
    for RatMat,users_vec,items_vec in zip(RatingMat_List,U_list,V_list):  
        TopN_pred = ials_ALLUSERSpred(RatMat,users_vec,items_vec,N) 
        All_TOPN_PRED.append(TopN_pred)
    return All_TOPN_PRED


In [ ]:
def getiALS_VUlist(UserItem_RatMAT,rank,iter,reg,alpha):
    V_list = []
    U_list = []
    for RatMAT in tqdm(UserItem_RatMAT):
        bst_Model = implicit.als.AlternatingLeastSquares(factors=rank, regularization=reg, iterations=iter,use_gpu = False)
        Item_UsersMAT = (RatMAT.T*alpha).astype('double')
        bst_Model.fit(Item_UsersMAT, show_progress=False)                
        user_vecs = bst_Model.user_factors
        item_vecs = bst_Model.item_factors
        V_list.append(item_vecs)
        U_list.append(user_vecs)
    return V_list,U_list 

In [ ]:
## Tuning::
## Tuning::
## Tuning::

In [ ]:
def HitrEval_noprint(Holdout,TopN_pred,user_column,item_column):
    Eval_itemsVector  =  Holdout[[item_column]].to_numpy()
    HitRate_arr   =  (TopN_pred == Eval_itemsVector).sum(axis=1)  ##sum along row...
    HitCount = np.count_nonzero(HitRate_arr == 1)
    HitRate_ = HitRate_arr.mean()
    return HitRate_

def TuneALS(train_set,holdout,als_param_grid):
    best_hr   = -np.inf
    best_model = None
    for i in trange(len(als_param_grid)):
        alpha, reg, rank, iter = list(als_param_grid)[i]  
        ALSmodel = implicit.als.AlternatingLeastSquares(
                   factors=rank, regularization=reg, iterations=iter,use_gpu = False)
        #train ALS model
        Item_UsersMAT = (train_set.T * alpha).astype('double')
        ALSmodel.fit(Item_UsersMAT, show_progress=False)                
        users_vecs = ALSmodel.user_factors
        items_vecs = ALSmodel.item_factors                                              
        ##############        ##############  users_vecs
        top10_pred = ials_TopNPred(train_set,holdout,users_vecs,items_vecs,'Updated_UserID', N=10) 
        test_hr = HitrEval_noprint(holdout,top10_pred,'Updated_UserID','Updated_ItemID')  #                           

        print('latent factors= {} ,reg_ = {}:, n_iter = {}, alpha = {}, HR= {}'.format(rank,reg,iter,alpha,test_hr))
        if test_hr > best_hr:
           best_hr  = test_hr
           best_rank = rank
           best_reg  = reg
           best_iter = iter
           best_alpha = alpha
           best_model = ALSmodel
    print('\n Best model; latent_factors= {} , reg_ = {}, n_iter = {}, alpha = {}, HR = {}'.format(best_rank,
                                                                              best_reg, best_iter,best_alpha,best_hr))
    return best_rank, best_reg, best_iter,best_alpha,best_hr,best_model    


# MovieLen_DF

In [ ]:
##MOVIElen_20M
newMLDF_20M = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/MLDF20M_afterUpdt.csv.zip')

In [ ]:
import pickle

with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/MLDF20M_newHoldout.pkl', 'rb') as f:
     newHoldoutlist_ml20 = pickle.load(f)     
    
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/MLDF20M_newUserItem.pkl', 'rb') as f:
     newUserItemlist_ml20 = pickle.load(f)


In [ ]:
rows_i = newMLDF_20M['Updated_UserID'].nunique()
cols_i = newMLDF_20M['Updated_ItemID'].nunique()
UserItemMATList_ml20 = AllRatingMatrices(newUserItemlist_ml20,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItemMATList_ml20))   
UserItemMATList_ml20

15


[<136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 8865107 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 8945455 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9025842 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9106051 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9186201 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9266468 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9346597 stored elements in Compressed Sparse Row format>,
 <136677x20720 sparse matrix of type '<class 'numpy.float64'>'
 	with 9426837 stored elements in Compres

In [ ]:
#### Tuning
####
#### Tuning

In [ ]:
rows_i = newMLDF_20M['Updated_UserID'].nunique()
cols_i = newMLDF_20M['Updated_ItemID'].nunique()

holdoutdf_ml20 = newHoldoutlist_ml20[0]
print("Holdout set: ",holdoutdf_ml20.shape[0])
print()
Trainmat_ml20 = SingleRatingMatrix(newUserItemlist_ml20[0],'Updated_UserID','Updated_ItemID',rows_i,cols_i)  #changes start within steps...
Trainmat_ml20

Holdout set:  3459



<136677x20720 sparse matrix of type '<class 'numpy.float64'>'
	with 8865107 stored elements in Compressed Sparse Row format>

In [ ]:
als_params = dict(alpha = [ 0.001,0.05, 0.1, 0.5, 1, 5, 10, 50, 100],
                  regularization = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 10],
                  rank = list(range(10, 81, 10)),
                  iter_array = [10,15,20,25,30])

als_param_grid, als_param_names = random_grid(als_params, n=60)
print(als_param_names)
len(als_param_grid)   

('alpha', 'regularization', 'rank', 'iter_array')


60

In [ ]:
for i in range(10):
    alpha, reg, rank, iter = list(als_param_grid)[i]  
    print(alpha, reg, rank,iter)

1 0.01 20 20
5 0.1 50 10
100 0.01 10 30
0.5 0.001 60 20
0.05 10 10 30
0.5 0.1 70 20
100 0.01 70 20
0.05 1 70 10
0.5 10 80 10
0.5 0.05 70 30


In [ ]:
bestModelInfo_ml20 = TuneALS(Trainmat_ml20,holdoutdf_ml20,als_param_grid)    

100%|██████████| 60/60 [1:36:04<00:00, 96.07s/it]latent factors= 20 ,reg_ = 0.5:, n_iter = 10, alpha = 0.05, HR= 0.0358485111303845

 Best model; latent_factors= 60 , reg_ = 0.1, n_iter = 20, alpha = 10, HR = 0.05608557386527898



In [ ]:
#latent_factors= 60 , reg_ = 0.1, n_iter = 20, alpha = 10, :
rank =  60
reglr = 0.01
iter =  20
alpha = 10

Vlist_ml20, Ulist_ml20 = getiALS_VUlist(UserItemMATList_ml20,rank=60,iter=20,reg=0.01,alpha=10)

100%|██████████| 15/15 [26:14<00:00, 104.99s/it]


In [ ]:
All_RTPRED_ml20 = ials_getALLPred(UserItemMATList_ml20,newHoldoutlist_ml20,Ulist_ml20,Vlist_ml20,'Updated_UserID',N=10) 
print(len(All_RTPRED_ml20))

15


In [ ]:
AllStepsRTHitr_ml20, LowerBand, Avg_HitRate, UpperBand   = getAll_HitRate(newHoldoutlist_ml20,All_RTPRED_ml20,'Updated_UserID','Updated_ItemID')

Number of hits:  199
Total Num of users:  3459
Recommendation HitRate:  0.05753107834634287
Number of hits:  188
Total Num of users:  3311
Recommendation HitRate:  0.05678042887345213
Number of hits:  166
Total Num of users:  3124
Recommendation HitRate:  0.05313700384122919
Number of hits:  198
Total Num of users:  3115
Recommendation HitRate:  0.06356340288924558
Number of hits:  185
Total Num of users:  3165
Recommendation HitRate:  0.05845181674565561
Number of hits:  152
Total Num of users:  3098
Recommendation HitRate:  0.04906391220142027
Number of hits:  161
Total Num of users:  3169
Recommendation HitRate:  0.050804670242978855
Number of hits:  187
Total Num of users:  3129
Recommendation HitRate:  0.05976350271652285
Number of hits:  229
Total Num of users:  3192
Recommendation HitRate:  0.07174185463659148
Number of hits:  217
Total Num of users:  3409
Recommendation HitRate:  0.06365503080082136
Number of hits:  198
Total Num of users:  3267
Recommendation HitRate:  0.06060

In [ ]:
AllStepsRTHitr_ml20

[0.05753107834634287,
 0.05678042887345213,
 0.05313700384122919,
 0.06356340288924558,
 0.05845181674565561,
 0.04906391220142027,
 0.050804670242978855,
 0.05976350271652285,
 0.07174185463659148,
 0.06365503080082136,
 0.06060606060606061,
 0.057523561910952224,
 0.06637806637806638,
 0.05635400907715583,
 0.05824175824175824]

In [ ]:
## MRR
AllStepsRTMRR_ml20, als_LBand, als_AvgMRR, als_UBand  = TQDMgetAll_MRR_Eval(newHoldoutlist_ml20,All_RTPRED_ml20,'Updated_ItemID')
AllStepsRTMRR_ml20

15it [00:00, 562.55it/s]


[0.021298,
 0.020325,
 0.018839,
 0.023648,
 0.022785,
 0.017929,
 0.017866,
 0.022683,
 0.027,
 0.021308,
 0.021749,
 0.019226,
 0.022571,
 0.022542,
 0.020212]

In [ ]:
## AllUsers Pred:  
AllUsrsRTPRED_ml20  = ialsALLPred_ALLUSERS(UserItemMATList_ml20,Ulist_ml20,Vlist_ml20,N=10)  
print(len(AllUsrsRTPRED_ml20))
AllUsrsRTPRED_ml20[:2]

15


[array([[  950,  1108,   140, ...,   108,  1129,   430],
        [   78,  1062,   663, ...,   105,   719,    15],
        [  179,   124,   249, ...,   135,   869,    29],
        ...,
        [ 6918,  6901,  6908, ...,  6903,  6902, 20719],
        [ 6918,  6901,  6908, ...,  6903,  6902, 20719],
        [ 6918,  6901,  6908, ...,  6903,  6902, 20719]]),
 array([[  108,   950,    81, ...,  1129,  1058,  1108],
        [   78,  1062,   969, ...,   719,   105,    15],
        [  179,   274,   199, ...,    22,   869,   112],
        ...,
        [ 6918,  6901,  6908, ...,  6903,  6902, 20719],
        [ 6918,  6901,  6908, ...,  6903,  6902, 20719],
        [ 6918,  6901,  6908, ...,  6903,  6902, 20719]])]

In [ ]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_20M/ML20_iRTnewPRED.pkl', 'wb') as f:  
     pickle.dump(AllUsrsRTPRED_ml20, f)

In [ ]:
##Coverage
alsAvgCovList_ml20 = AllSteps_Coverage_Ratio(newMLDF_20M,AllUsrsRTPRED_ml20,'Updated_ItemID')
alsAvgCovList_ml20

[0.098456,
 0.098842,
 0.098745,
 0.098986,
 0.099469,
 0.100531,
 0.101014,
 0.100917,
 0.10111,
 0.1014,
 0.101641,
 0.102413,
 0.102268,
 0.102751,
 0.102268]

In [ ]:
AllSteps  = list(range(1,15))
iALSCorr_ml20 = Updt_getAll_AvgCorr(AllUsrsRTPRED_ml20,newUserItemlist_ml20,AllSteps,'Updated_UserID')  #,
for df in iALSCorr_ml20:
    print(df.mean()) 

0.5819352882884975
0.5667725133848397
0.5684140172124739
0.5803752847244131
0.5827544460078844
0.5814161123003696
0.5915663562648992
0.5698909614231138
0.5739098158109834
0.5751296519964466
0.5868151198456625
0.5859081555069298
0.5834152388368218
0.5849117769752793


# Amz_Beauty

In [ ]:
#AMZBty
newAMZB_DF = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_afterUpdt.csv')

In [ ]:
import pickle

with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_newHoldout.pkl', 'rb') as f:
     newHoldoutlist_amb = pickle.load(f)     
    
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_newUserItem.pkl', 'rb') as f:
     newUserItemlist_amb = pickle.load(f)

In [ ]:
rows_i = newAMZB_DF['Updated_UserID'].nunique()
cols_i = newAMZB_DF['Updated_ItemID'].nunique()
UserItem_MATList_amb = AllRatingMatrices(newUserItemlist_amb,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItem_MATList_amb))   
UserItem_MATList_amb

10


[<22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 40228 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 41330 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 42432 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 43525 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 44665 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 45752 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 47034 stored elements in Compressed Sparse Row format>,
 <22557x8109 sparse matrix of type '<class 'numpy.float64'>'
 	with 48112 stored elements in Compressed Sparse Row format>,
 <22557x

In [ ]:
#### Tuning
###
#### Tuning

In [ ]:
rows_i = newAMZB_DF['Updated_UserID'].nunique()
cols_i = newAMZB_DF['Updated_ItemID'].nunique()

holdout_df = newHoldoutlist_amb[0]
print("Holdout set: ",holdout_df.shape[0])
print()
Train_mat = SingleRatingMatrix(newUserItemlist_amb[0],'Updated_UserID','Updated_ItemID',rows_i,cols_i)  #changes start within steps...
Train_mat

Holdout set:  474



<22557x8109 sparse matrix of type '<class 'numpy.float64'>'
	with 40228 stored elements in Compressed Sparse Row format>

In [ ]:
als_params = dict(alpha = [ 0.001,0.05, 0.1, 0.5, 1, 5, 10, 50, 100],
                  regularization = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 10],
                  rank = list(range(10, 81, 10)),
                  iter_array = [10,15,20,25,30])

als_param_grid, als_param_names = random_grid(als_params, n=60)
print(als_param_names)
len(als_param_grid)   

('alpha', 'regularization', 'rank', 'iter_array')


60

In [ ]:
for i in range(10):
    alpha, reg, rank, iter = list(als_param_grid)[i]  
    print(alpha, reg, rank,iter)

1 1 50 20
50 1 70 15
5 0.05 60 15
0.5 0.5 70 25
50 0.005 10 30
5 0.001 10 10
0.1 10 40 20
100 0.5 70 25
5 0.001 70 15
10 0.05 50 20


In [ ]:
bestModel_Info = TuneALS(Train_mat,holdout_df,als_param_grid)    

100%|██████████| 60/60 [00:50<00:00,  1.19it/s]latent factors= 40 ,reg_ = 0.5:, n_iter = 20, alpha = 10, HR= 0.31856540084388185

 Best model; latent_factors= 70 , reg_ = 0.5, n_iter = 25, alpha = 100, HR = 0.3438818565400844



In [ ]:
##############
# iALS Retrain
############## 

In [ ]:
#latent_factors= 70 , reg_ = 0.5, n_iter = 25, alpha = 100,   BEST_new
rank =  70
reglr = 0.5
iter =  25
alpha = 100

In [ ]:
V_list_bty,U_list_bty = getiALS_VUlist(UserItem_MATList_amb,rank=70,iter=25,reg=0.5,alpha=100)

100%|██████████| 10/10 [00:17<00:00,  1.78s/it]


In [ ]:
##Hitrate
top10_pred = ials_TopNPred(UserItem_MATList_amb[0],newHoldoutlist_amb[0],U_list_bty[0],V_list_bty[0],'Updated_UserID', N=10)
print(top10_pred.shape)
top10_pred

(474, 10)


array([[ 995,   22,  483, ..., 2221, 2852, 1061],
       [ 294,  309, 1283, ..., 1160,  920,  939],
       [3062,  371, 2245, ..., 4349,  919, 6098],
       ...,
       [ 239,  251,  319, ..., 3030,  469,  551],
       [ 815,  251,  319, ..., 1126, 1099,  871],
       [ 369,  420,  426, ...,  263,  231,  144]])

In [ ]:
#
Hitrate_Eval(newHoldoutlist_amb[0],top10_pred,'Updated_UserID','Updated_ItemID')  #

Number of hits:  162
Total Num of users:  474
Recommendation HitRate:  0.34177215189873417


0.34177215189873417

In [ ]:
All_ialsPRED_bty = ials_getALLPred(UserItem_MATList_amb,newHoldoutlist_amb,U_list_bty,V_list_bty,'Updated_UserID',N=10) 
print(len(All_ialsPRED_bty))

10


In [ ]:
AllStepsHitr_iALS_bty, LowerBand, Avg_HitRate, UpperBand   = getAll_HitRate(newHoldoutlist_amb,All_ialsPRED_bty,'Updated_UserID','Updated_ItemID')

Number of hits:  162
Total Num of users:  474
Recommendation HitRate:  0.34177215189873417
Number of hits:  174
Total Num of users:  433
Recommendation HitRate:  0.4018475750577367
Number of hits:  194
Total Num of users:  465
Recommendation HitRate:  0.4172043010752688
Number of hits:  179
Total Num of users:  478
Recommendation HitRate:  0.37447698744769875
Number of hits:  173
Total Num of users:  478
Recommendation HitRate:  0.3619246861924686
Number of hits:  169
Total Num of users:  508
Recommendation HitRate:  0.33267716535433073
Number of hits:  272
Total Num of users:  612
Recommendation HitRate:  0.4444444444444444
Number of hits:  230
Total Num of users:  599
Recommendation HitRate:  0.38397328881469117
Number of hits:  205
Total Num of users:  536
Recommendation HitRate:  0.3824626865671642
Number of hits:  117
Total Num of users:  425
Recommendation HitRate:  0.2752941176470588
Average HitRate for All Recommendations:  0.3716077404499597


In [ ]:
AllStepsHitr_iALS_bty

[0.34177215189873417,
 0.4018475750577367,
 0.4172043010752688,
 0.37447698744769875,
 0.3619246861924686,
 0.33267716535433073,
 0.4444444444444444,
 0.38397328881469117,
 0.3824626865671642,
 0.2752941176470588]

In [ ]:
#MRR
als_AllStepsMRR_bty, als_LBand, als_AvgMRR, als_UBand  = TQDMgetAll_MRR_Eval(newHoldoutlist_amb,All_ialsPRED_bty,'Updated_ItemID')
als_AllStepsMRR_bty

10it [00:00, 625.32it/s]


[0.311412,
 0.371047,
 0.368763,
 0.34477,
 0.31446,
 0.289084,
 0.318045,
 0.321354,
 0.336434,
 0.214555]

In [ ]:
##All USERS prediction:
AllUsrSPRED_ambRT  = ialsALLPred_ALLUSERS(UserItem_MATList_amb,U_list_bty,V_list_bty,N=10)  
print(len(AllUsrSPRED_ambRT))
AllUsrSPRED_ambRT[:2]

10


[array([[  71,  116,    1, ...,  147,   21,    2],
        [  60,   43,   71, ...,  116,   62,  215],
        [ 297,   60,   43, ...,    2,  215,  116],
        ...,
        [2702, 2699, 2695, ..., 2705, 2706, 8108],
        [2702, 2699, 2695, ..., 2705, 2706, 8108],
        [2702, 2699, 2695, ..., 2705, 2706, 8108]]),
 array([[  71,  134,  116, ...,   21,  297,  138],
        [  60,   43,  297, ...,    9,   21,  215],
        [ 297,    3,   60, ...,  215,   21,   43],
        ...,
        [2702, 2699, 2695, ..., 2705, 2706, 8108],
        [2702, 2699, 2695, ..., 2705, 2706, 8108],
        [2702, 2699, 2695, ..., 2705, 2706, 8108]])]

In [ ]:
# with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_Btyfiles/AMZB_ialsRT_allPRED.pkl', 'wb') as f:  
#      pickle.dump(All_usersPRED_amb, f)

In [ ]:
##Coverage
als_AvgCovList_bty = AllSteps_Coverage_Ratio(newAMZB_DF,AllUsrSPRED_ambRT,'Updated_ItemID')
als_AvgCovList_bty

[0.51671,
 0.530892,
 0.534221,
 0.541374,
 0.54347,
 0.553829,
 0.565544,
 0.575163,
 0.571217,
 0.574793]

In [ ]:
AllSteps  = list(range(1,10))
iALSCorr_ = Updt_getAll_AvgCorr(AllUsrSPRED_ambRT,newUserItemlist_amb,AllSteps,'Updated_UserID')  #,

iALSCorr_.shape
for df in iALSCorr_:
    print(df.mean())  

0.3731984009681043
0.37452346353959165
0.37134439823009313
0.3600623158271102
0.3689771706730944
0.3721809498378752
0.38422220311050215
0.36200365056613304
0.37840815342348627


# MovieLen_1M

In [ ]:
newMLDF_1M = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/MLDF1M_afterUpdt.csv.zip')
print(newMLDF_1M.shape)   #userId_int

(575281, 9)


In [ ]:
import pickle

with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/MLDF1M_newHoldout.pkl', 'rb') as f:
     newHoldout_listML1 = pickle.load(f)     
    
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/MLDF1M_newUserItem.pkl', 'rb') as f:
     newUserItem_listML1 = pickle.load(f)


In [ ]:
rows_i = newMLDF_1M['Updated_UserID'].nunique()
cols_i = newMLDF_1M['Updated_ItemID'].nunique()
UserItemMATList_ML1 = AllRatingMatrices(newUserItem_listML1,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItemMATList_ML1))   
UserItemMATList_ML1

10


[<6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 500332 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 508812 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 517278 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 525757 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 534098 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 542538 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 551048 stored elements in Compressed Sparse Row format>,
 <6038x3533 sparse matrix of type '<class 'numpy.float64'>'
 	with 559597 stored elements in Compressed Sparse Row format>,
 <6038x3

In [ ]:
######################
# Tuning
######################

In [ ]:
rows_i = newMLDF_1M['Updated_UserID'].nunique()
cols_i = newMLDF_1M['Updated_ItemID'].nunique()

holdoutdf_ml1 = newHoldout_listML1[0]
print("Holdout set: ",holdoutdf_ml1.shape[0])
print()
Trainmat_ml1 = SingleRatingMatrix(newUserItem_listML1[0],'Updated_UserID','Updated_ItemID',rows_i,cols_i)  #changes start within steps...
Trainmat_ml1

Holdout set:  233



<6038x3533 sparse matrix of type '<class 'numpy.float64'>'
	with 500332 stored elements in Compressed Sparse Row format>

In [ ]:
als_params = dict(alpha = [ 0.001,0.05, 0.1, 0.5, 1, 5, 10, 50, 100],
                  regularization = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 10],
                  rank = list(range(10, 81, 10)),
                  iter_array = [10,15,20,25,30])

als_param_grid, als_param_names = random_grid(als_params, n=60)
print(als_param_names)
len(als_param_grid)   

('alpha', 'regularization', 'rank', 'iter_array')


60

In [ ]:
for i in range(10):
    alpha, reg, rank, iter = list(als_param_grid)[i]  
    print(alpha, reg, rank,iter)

0.001 10 20 30
0.1 0.01 20 15
0.05 0.05 70 25
100 0.01 50 20
1 0.001 80 30
0.05 0.5 20 10
10 0.001 60 20
50 1 30 15
50 0.001 70 10
100 0.001 60 15


In [ ]:
bestModelInfo_ml1 = TuneALS(Trainmat_ml1,holdoutdf_ml1,als_param_grid)    

100%|██████████| 60/60 [05:26<00:00,  5.45s/it]latent factors= 30 ,reg_ = 0.005:, n_iter = 15, alpha = 50, HR= 0.04291845493562232

 Best model; latent_factors= 80 , reg_ = 0.05, n_iter = 30, alpha = 1, HR = 0.08583690987124463



In [ ]:
# Best model; latent_factors= 80 , reg_ = 0.05, n_iter = 30, alpha = 1 
rank =  80
reglr = 0.05
iter =  30
alpha = 1

Vlist_ml1,Ulist_ml1 = getiALS_VUlist(UserItemMATList_ML1, rank=80, iter=30, reg=0.05,alpha= 1)

100%|██████████| 10/10 [02:33<00:00, 15.35s/it]


In [ ]:
All_RTPRED_ml1 = ials_getALLPred(UserItemMATList_ML1,newHoldout_listML1,Ulist_ml1,Vlist_ml1,'Updated_UserID',N=10) 
print(len(All_RTPRED_ml1))
AllStepsRTHitr_ml1, LowerBand, Avg_HitRate, UpperBand   = getAll_HitRate(newHoldout_listML1,All_RTPRED_ml1,'Updated_UserID','Updated_ItemID')

10
Number of hits:  15
Total Num of users:  233
Recommendation HitRate:  0.06437768240343347
Number of hits:  18
Total Num of users:  255
Recommendation HitRate:  0.07058823529411765
Number of hits:  28
Total Num of users:  286
Recommendation HitRate:  0.0979020979020979
Number of hits:  29
Total Num of users:  309
Recommendation HitRate:  0.09385113268608414
Number of hits:  23
Total Num of users:  468
Recommendation HitRate:  0.049145299145299144
Number of hits:  25
Total Num of users:  526
Recommendation HitRate:  0.04752851711026616
Number of hits:  27
Total Num of users:  517
Recommendation HitRate:  0.05222437137330754
Number of hits:  20
Total Num of users:  470
Recommendation HitRate:  0.0425531914893617
Number of hits:  25
Total Num of users:  453
Recommendation HitRate:  0.05518763796909492
Number of hits:  21
Total Num of users:  362
Recommendation HitRate:  0.058011049723756904
Average HitRate for All Recommendations:  0.06313692150968195


In [ ]:
AllStepsRTHitr_ml1

[0.06437768240343347,
 0.07058823529411765,
 0.0979020979020979,
 0.09385113268608414,
 0.049145299145299144,
 0.04752851711026616,
 0.05222437137330754,
 0.0425531914893617,
 0.05518763796909492,
 0.058011049723756904]

In [ ]:
## MRR
AllStepsRTMRR_ml1, als_LBand, als_AvgMRR, als_UBand  = TQDMgetAll_MRR_Eval(newHoldout_listML1,All_RTPRED_ml1,'Updated_ItemID')
AllStepsRTMRR_ml1

10it [00:00, 605.50it/s]


[0.021006,
 0.023399,
 0.035424,
 0.034254,
 0.013204,
 0.01421,
 0.025114,
 0.016873,
 0.022615,
 0.016527]

In [ ]:
## AllUsers Pred:  
AllUsrsRTPRED_ml1  = ialsALLPred_ALLUSERS(UserItemMATList_ML1, Ulist_ml1, Vlist_ml1, N=10)  
print(len(AllUsrsRTPRED_ml1))
AllUsrsRTPRED_ml1[:2]

10


[array([[ 134,  336,  256, ...,  242,  138,  641],
        [ 563,  140,  908, ...,  369,  587,  134],
        [ 219,  223,  259, ...,  460,   23,  171],
        ...,
        [1181, 1179, 1182, ..., 1177, 1176, 3532],
        [1181, 1179, 1182, ..., 1177, 1176, 3532],
        [1181, 1179, 1182, ..., 1177, 1176, 3532]]),
 array([[ 336,  134,  256, ...,  385,  114,  641],
        [ 140,  563,  908, ...,  369,  398,  587],
        [ 219,  259,  223, ...,  156,   54,  585],
        ...,
        [1181, 1179, 1182, ..., 1177, 1176, 3532],
        [1181, 1179, 1182, ..., 1177, 1176, 3532],
        [1181, 1179, 1182, ..., 1177, 1176, 3532]])]

In [ ]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/ML_1M/ML1iRT_newPRED.pkl', 'wb') as f:  
     pickle.dump(AllUsrsRTPRED_ml1, f)

In [ ]:
###Coverage ||.....||
alsRTCovList_ml1 = AllSteps_Coverage_Ratio(newMLDF_1M,AllUsrsRTPRED_ml1,'Updated_ItemID')
alsRTCovList_ml1

[0.230682,
 0.230682,
 0.234079,
 0.23238,
 0.233513,
 0.234362,
 0.233513,
 0.234079,
 0.23606,
 0.237475]

In [ ]:
AllSteps  = list(range(1,10))
iALSCorr_ml1 = Updt_getAll_AvgCorr(AllUsrsRTPRED_ml1,newUserItem_listML1,AllSteps,'Updated_UserID')  #,
for df in iALSCorr_ml1:
    print(df.mean()) 

0.6396744045164181
0.6611167582446449
0.6488851144620298
0.6346426488938057
0.6266215250864838
0.6487937150831147
0.643720971434931
0.6386380717685826
0.6467061937399258


#Amz_Games

In [ ]:
newAMZGames_DF = pd.read_csv('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZGames_afterUpdt.csv')

In [ ]:
##lOAD Saved Lists...
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZGames_newUserItem.pkl', 'rb') as f:   #
     newUserItemlist_AMZg = pickle.load(f)

with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZGames_newHoldout.pkl', 'rb') as f:
     newHOLDOUTLIST_AMZg  = pickle.load(f)


In [ ]:
rows_i = newAMZGames_DF['Updated_UserID'].nunique()
cols_i = newAMZGames_DF['Updated_ItemID'].nunique()
UserItemMATList_AMZg = AllRatingMatrices(newUserItemlist_AMZg,'Updated_UserID','Updated_ItemID',rows_i ,cols_i) 
print(len(UserItemMATList_AMZg))   
UserItemMATList_AMZg

15


[<39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 250384 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 255520 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 261105 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 266597 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 272102 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 277572 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 282899 stored elements in Compressed Sparse Row format>,
 <39284x17318 sparse matrix of type '<class 'numpy.float64'>'
 	with 288493 stored elements in Compressed Sparse Row f

In [ ]:
################################
# Tuning
################################

In [ ]:
rows_i = newAMZGames_DF['Updated_UserID'].nunique()
cols_i = newAMZGames_DF['Updated_ItemID'].nunique()

holdoutdf_AMZg = newHOLDOUTLIST_AMZg[0]
print("Holdout set: ",holdoutdf_AMZg.shape[0])
print()
Trainmat_AMZg = SingleRatingMatrix(newUserItemlist_AMZg[0],'Updated_UserID','Updated_ItemID',rows_i,cols_i)  #changes start within steps...
Trainmat_AMZg

Holdout set:  2018



<39284x17318 sparse matrix of type '<class 'numpy.float64'>'
	with 250384 stored elements in Compressed Sparse Row format>

In [ ]:
als_params = dict(alpha = [ 0.001,0.05, 0.1, 0.5, 1, 5, 10, 50, 100],
                  regularization = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 10],
                  rank = list(range(10, 81, 10)),
                  iter_array = [10,15,20,25,30])

als_param_grid, als_param_names = random_grid(als_params, n=60)
print(als_param_names)
len(als_param_grid) 

('alpha', 'regularization', 'rank', 'iter_array')


60

In [ ]:
for i in range(10):
    alpha, reg, rank, iter = list(als_param_grid)[i]  
    print(alpha, reg, rank,iter)

1 0.01 20 20
0.1 10 80 10
0.5 0.05 50 30
10 0.01 30 25
100 0.005 60 10
1 0.001 30 20
50 10 50 10
0.1 0.001 30 20
10 1 50 10
0.05 0.5 80 30


In [ ]:
bestModelInfo_AMZg = TuneALS(Trainmat_AMZg,holdoutdf_AMZg,als_param_grid)    

100%|██████████| 60/60 [03:53<00:00,  3.89s/it]latent factors= 50 ,reg_ = 1:, n_iter = 25, alpha = 50, HR= 0.06342913776015857

 Best model; latent_factors= 80 , reg_ = 0.5, n_iter = 25, alpha = 50, HR = 0.06788899900891972



In [ ]:
#bestModel: latent_factors= 80 , reg_ = 0.5, n_iter = 25, alpha = 50,
rank =  80
reglr = 0.5
iter =  25
alpha = 50

Vlist_amzg,Ulist_amzg = getiALS_VUlist(UserItemMATList_AMZg,rank=80,iter=25,reg=0.5,alpha=50)

100%|██████████| 15/15 [01:53<00:00,  7.58s/it]


In [ ]:
TestUsrsPRED_amzg = ials_getALLPred(UserItemMATList_AMZg, newHOLDOUTLIST_AMZg, Ulist_amzg, Vlist_amzg,'Updated_UserID',N=10) 
print(len(TestUsrsPRED_amzg))
AllStepsRTHitr_amzg, LowerBand, Avg_HitRate, UpperBand = getAll_HitRate(newHOLDOUTLIST_AMZg,TestUsrsPRED_amzg,'Updated_UserID','Updated_ItemID')

15
Number of hits:  142
Total Num of users:  2018
Recommendation HitRate:  0.07036669970267592
Number of hits:  151
Total Num of users:  2200
Recommendation HitRate:  0.06863636363636363
Number of hits:  160
Total Num of users:  2172
Recommendation HitRate:  0.07366482504604052
Number of hits:  175
Total Num of users:  2079
Recommendation HitRate:  0.08417508417508418
Number of hits:  168
Total Num of users:  2089
Recommendation HitRate:  0.08042125418860699
Number of hits:  158
Total Num of users:  2120
Recommendation HitRate:  0.07452830188679245
Number of hits:  132
Total Num of users:  2217
Recommendation HitRate:  0.05953991880920163
Number of hits:  125
Total Num of users:  2207
Recommendation HitRate:  0.05663797009515179
Number of hits:  157
Total Num of users:  2133
Recommendation HitRate:  0.0736052508204407
Number of hits:  171
Total Num of users:  2171
Recommendation HitRate:  0.0787655458314141
Number of hits:  162
Total Num of users:  2261
Recommendation HitRate:  0.07164

In [ ]:
AllStepsRTHitr_amzg

[0.07036669970267592,
 0.06863636363636363,
 0.07366482504604052,
 0.08417508417508418,
 0.08042125418860699,
 0.07452830188679245,
 0.05953991880920163,
 0.05663797009515179,
 0.0736052508204407,
 0.0787655458314141,
 0.07164971251658558,
 0.07524487978628673,
 0.0714936247723133,
 0.0749228735125606,
 0.05715571557155716]

In [ ]:
## MRR
AllStepsRTMRR_amzg, als_LBand, als_AvgMRR, als_UBand  = TQDMgetAll_MRR_Eval(newHOLDOUTLIST_AMZg,TestUsrsPRED_amzg,'Updated_ItemID')
AllStepsRTMRR_amzg

15it [00:00, 566.29it/s]


[0.029182,
 0.0291,
 0.027641,
 0.03636,
 0.034335,
 0.034739,
 0.029932,
 0.023984,
 0.034718,
 0.032746,
 0.028055,
 0.030768,
 0.02987,
 0.034885,
 0.023542]

In [ ]:
## AllUsersPred:::  
AllUsrsRTPRED_amzg  = ialsALLPred_ALLUSERS(UserItemMATList_AMZg,Ulist_amzg,Vlist_amzg,N=10)  
print(len(AllUsrsRTPRED_amzg))
AllUsrsRTPRED_amzg[:2]

15


[array([[  385,   145,   825, ...,   916,   204,   380],
        [14760,    29, 14555, ...,   874,   138, 13908],
        [  346,   687,   385, ...,   181,   610,   155],
        ...,
        [ 5775,  5777,  5782, ...,  5771,  5770, 17317],
        [ 5775,  5777,  5782, ...,  5771,  5770, 17317],
        [ 5775,  5777,  5782, ...,  5771,  5770, 17317]]),
 array([[  145,   385,   825, ...,  1434,   916,   380],
        [11924, 13491, 14555, ..., 14760,    23, 13171],
        [  346,   174,   687, ...,     0,   705,   613],
        ...,
        [ 5775,  5777,  5782, ...,  5771,  5770, 17317],
        [ 5775,  5777,  5782, ...,  5771,  5770, 17317],
        [ 5775,  5777,  5782, ...,  5771,  5770, 17317]])]

In [ ]:
with open('/mnt/bulky/oolaleke/TDM_Dataset/AMZ_GamesFiles/AMZG_iRTnewPRED.pkl', 'wb') as f:  
     pickle.dump(AllUsrsRTPRED_amzg, f)

In [ ]:
## Coverage:::
alsRTCovList_AMZg = AllSteps_Coverage_Ratio(newAMZGames_DF, AllUsrsRTPRED_amzg, 'Updated_ItemID')
alsRTCovList_AMZg 

[0.191304,
 0.194422,
 0.193556,
 0.194191,
 0.194595,
 0.194768,
 0.19344,
 0.193498,
 0.195173,
 0.194364,
 0.193671,
 0.194307,
 0.194884,
 0.192228,
 0.194942]

In [ ]:
AllSteps  = list(range(1,15))
iALSCorr_amzg = Updt_getAll_AvgCorr(AllUsrsRTPRED_amzg, newUserItemlist_AMZg, AllSteps,'Updated_UserID') #
for df in iALSCorr_amzg:
    print(df.mean()) 

0.42018598742262653
0.42555449747577945
0.4232726066488098
0.417317937330896
0.41799430819152406
0.4237161069162328
0.4288281630852342
0.4247635511827896
0.42661200775151426
0.4280036882104114
0.42405788968832886
0.42877834170285967
0.4276171502777493
0.4288725756438257
